In [1]:
import pandas as pd 
import numpy as np

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
aggregate = [
  {
    "$match": {
      "data_description.project_name": "NP Ultra and Psychedelics",
      "data_description.data_level": "derived"
    }
  },
  {
    "$project": {
      "name": 1, 
      "subject_id": "$data_description.subject_id",
      "genotype": "$subject.genotype", 
      "date_of_birth": "$subject.date_of_birth", 
      "sex": "$subject.sex", 
      "session_type": "$session.session_type",
      "session_time": "$session.session_start_time",
      "stimulus_epochs": "$session.stimulus_epochs.stimulus_name",  
      "project_name": "$data_description.project_name", 
      "modality": "$data_description.modality.name",
      "notes": "$session.stimulus_epochs.notes"
    }
  }
]
records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [4]:
df = pd.DataFrame(records)
df.head()

,_id,name,subject_id,genotype,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,notes
0,916393f2-7009-421a-bde7-bbaf95416937,ecephys_714527_2024-05-14_13-22-07_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,A study on the effect of psilocybin on neocort...,2024-05-14T10:25:34.114677+00:00,"[OptoTagging, OptoTagging, OptoTagging, RFMapp...",NP Ultra and Psychedelics,[Extracellular electrophysiology],[Saline injection occurred ~5 min after this s...
1,c867ad1e-bd84-42df-b863-1244a2fcc957,ecephys_714789_2024-05-16_13-16-59_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,A study on the effect of psilocybin on neocort...,2024-05-16T10:26:41.826355+00:00,"[OptoTagging, OptoTagging, OptoTagging, RFMapp...",NP Ultra and Psychedelics,[Extracellular electrophysiology],[Saline injection occured ~1 min after this st...
2,d389dfbd-257a-4b19-b55f-e6c75208f1ec,ecephys_730049_2024-06-13_12-53-21_nwb_2025-08...,730049,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2024-03-09,Female,A study on the effect of psilocybin on neocort...,2024-06-13T09:56:19.252728+00:00,"[OptoTagging, OptoTagging, OptoTagging, OptoTa...",NP Ultra and Psychedelics,[Extracellular electrophysiology],[Saline injection occurred ~1 minute after thi...
3,8a00ee0c-c470-43c5-b24c-52c1785bee3c,ecephys_714527_2024-05-15_13-00-23_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,A study on the effect of psilocybin on neocort...,2024-05-15T10:43:35.535979+00:00,"[OptoTagging, OptoTagging, OptoTagging, RFMapp...",NP Ultra and Psychedelics,[Extracellular electrophysiology],[Psilocybin injection (3 mg/kg) occurred ~1 mi...
4,48c07938-c41b-4cd6-b9e6-c25c0a90aedb,ecephys_714789_2024-05-17_11-47-07_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,A study on the effect of psilocybin on neocort...,2024-05-17T09:26:22.868269+00:00,"[OptoTagging, OptoTagging, OptoTagging, RFMapp...",NP Ultra and Psychedelics,[Extracellular electrophysiology],[Psilocybin injection (3 mg/kg) occurred ~1 mi...


In [5]:
# Reshape the dataframe and inputs (since metadata is incomplete) 

df = df.sort_values(by = 'session_time')
df['experimental_purpose'] = df['session_type']
df['session_type'] = ['saline','psilocybin']*len(df['subject_id'].unique())
df['modality'] = [v[0] for v in df['modality']]

saline_epochs = [
    'Spontaneous_0','RFMapping_0','OptoTagging_0','Injection',
    'Spontaneous_1','RFMapping_1','OptoTagging_1',
    'Spontaneous_2','RFMapping_2','OptoTagging_2', 'Anesthesia', 
    'Spontaneous_3','RFMapping_3','Spontaneous_4'
]
psilo_epochs = [
    'Spontaneous_0','RFMapping_0','OptoTagging_0','Injection',
    'Spontaneous_1','RFMapping_1','OptoTagging_1',
    'Spontaneous_2','RFMapping_2','OptoTagging_2'
]

df['stimulus_epochs'] = [saline_epochs,psilo_epochs]*len(df['subject_id'].unique())

df['stimulus_types'] = list([np.unique([s.split('_')[0] for s in df['stimulus_epochs'].iloc[0]]).astype(str)])*len(df)
df['stimulus_types'] = [v['stimulus_types'].astype(list) for idx,v in df.iterrows()]

df.head()

,_id,name,subject_id,genotype,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,notes,experimental_purpose,stimulus_types
0,916393f2-7009-421a-bde7-bbaf95416937,ecephys_714527_2024-05-14_13-22-07_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,saline,2024-05-14T10:25:34.114677+00:00,"[Spontaneous_0, RFMapping_0, OptoTagging_0, In...",NP Ultra and Psychedelics,Extracellular electrophysiology,[Saline injection occurred ~5 min after this s...,A study on the effect of psilocybin on neocort...,"[Anesthesia, Injection, OptoTagging, RFMapping..."
3,8a00ee0c-c470-43c5-b24c-52c1785bee3c,ecephys_714527_2024-05-15_13-00-23_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,psilocybin,2024-05-15T10:43:35.535979+00:00,"[Spontaneous_0, RFMapping_0, OptoTagging_0, In...",NP Ultra and Psychedelics,Extracellular electrophysiology,[Psilocybin injection (3 mg/kg) occurred ~1 mi...,A study on the effect of psilocybin on neocort...,"[Anesthesia, Injection, OptoTagging, RFMapping..."
1,c867ad1e-bd84-42df-b863-1244a2fcc957,ecephys_714789_2024-05-16_13-16-59_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,saline,2024-05-16T10:26:41.826355+00:00,"[Spontaneous_0, RFMapping_0, OptoTagging_0, In...",NP Ultra and Psychedelics,Extracellular electrophysiology,[Saline injection occured ~1 min after this st...,A study on the effect of psilocybin on neocort...,"[Anesthesia, Injection, OptoTagging, RFMapping..."
4,48c07938-c41b-4cd6-b9e6-c25c0a90aedb,ecephys_714789_2024-05-17_11-47-07_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,psilocybin,2024-05-17T09:26:22.868269+00:00,"[Spontaneous_0, RFMapping_0, OptoTagging_0, In...",NP Ultra and Psychedelics,Extracellular electrophysiology,[Psilocybin injection (3 mg/kg) occurred ~1 mi...,A study on the effect of psilocybin on neocort...,"[Anesthesia, Injection, OptoTagging, RFMapping..."
5,f6792a6f-e70b-4d42-b0b0-9007adc65775,ecephys_717033_2024-06-04_13-01-40_nwb_2025-08...,717033,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,2023-12-18,Male,saline,2024-06-04T10:11:23.881625+00:00,"[Spontaneous_0, RFMapping_0, OptoTagging_0, In...",NP Ultra and Psychedelics,Extracellular electrophysiology,[Saline injection occurred ~1 minute after the...,A study on the effect of psilocybin on neocort...,"[Anesthesia, Injection, OptoTagging, RFMapping..."


In [7]:
df.to_csv('/data/metadata/npultra_metadata.csv', index= False)